## Bi-LSTM?
- Bidirectional RNN model
- able to preserve information from both past and future
- consists of two networks, one access pastinformation in forward direction and another access future in the reverse direction

*While LSTM only allows to store information from the past, Bi-LSTM can access both past and future so it allows better understanding in context.*


## Practice Code : Make a sentence

In [1]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

In [6]:
sentences = ('Lorem ipsum dolor sit amet consectetur adipisicing elit '
    'sed do eiusmod tempor incididunt ut labore et dolore magna '
    'aliqua Ut enim ad minim veniam quis nostrud exercitation')

# {'quis': 0, 'magna': 1, 'aliqua': 2, 'adipisicing': 3, ...}
word_dict = {w : i for i,w in enumerate(list(set(sentences.split())))}
# {0: 'quis', 1: 'magna', 2: 'aliqua', 3: 'adipisicing'...}
number_dict = {i : w for i,w in enumerate(list(set(sentences.split())))}
n_class = len(word_dict)
n_step = len(sentences.split())
n_hidden = 5

In [12]:
def make_batch(sentence):
    input_batch = []
    target_batch = []

    words = sentence.split()
    for i, word in enumerate(words[:-1]):
        input = [word_dict[n] for n in words[:(i + 1)]]  # getting indexes accumulatively
        input = input + [0] * (n_step - len(input)) # fill out the rest indexes with 0s
        target = word_dict[words[i + 1]] # getting next index of input
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(np.eye(n_class)[target])

    return input_batch, target_batch

In [15]:
# Bi_LSTM model
X = tf.placeholder(tf.float32, [None, n_step, n_class])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden*2, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

# since it's bidirectional, include both forward and backward
lstm_fw_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)
lstm_bw_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)

# outputs : [batch_size, len_seq, n_hidden], states : [batch_size, n_hidden]
outputs, _ = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, X, dtype=tf.float32)

outputs = tf.concat([outputs[0], outputs[1]],2) # output[0]: lstm_tw, output[1]: lstm_bw
outputs = tf.transpose(outputs,[1,0,2])  # [n_step, batch_size, n_hidden]
outputs = outputs[-1] # [batch_size, n_hidden]

model = tf.matmul(outputs,W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

prediction = tf.cast(tf.argmax(model,1), tf.int32)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [19]:
# Training
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

input_batch, output_batch = make_batch(sentences)

for epoch in range(10000):
    _, loss = sess.run([optimizer, cost], feed_dict = {X: input_batch, Y: output_batch})
    if (epoch+1)%500 ==0:
        print('Epoch: ', '%04d' % (epoch+1),'cost=', '{:.6f}'.format(loss))
        
predict = sess.run([prediction], feed_dict={X: input_batch})
print('original :' , sentences)
print('result :', [number_dict[n] for n in [pre for pre in predict[0]]])

Epoch:  0500 cost= 1.980381
Epoch:  1000 cost= 1.442016
Epoch:  1500 cost= 1.090175
Epoch:  2000 cost= 0.849018
Epoch:  2500 cost= 0.673616
Epoch:  3000 cost= 0.538919
Epoch:  3500 cost= 0.424251
Epoch:  4000 cost= 0.344240
Epoch:  4500 cost= 0.291671
Epoch:  5000 cost= 0.262796
Epoch:  5500 cost= 0.232523
Epoch:  6000 cost= 0.190519
Epoch:  6500 cost= 0.152863
Epoch:  7000 cost= 0.133099
Epoch:  7500 cost= 0.119266
Epoch:  8000 cost= 0.107849
Epoch:  8500 cost= 0.098265
Epoch:  9000 cost= 2.787060
Epoch:  9500 cost= 0.088820
Epoch:  10000 cost= 0.085580
original : Lorem ipsum dolor sit amet consectetur adipisicing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua Ut enim ad minim veniam quis nostrud exercitation
result : ['ipsum', 'dolor', 'sit', 'amet', 'consectetur', 'adipisicing', 'elit', 'sed', 'do', 'eiusmod', 'tempor', 'incididunt', 'ut', 'labore', 'et', 'dolore', 'magna', 'aliqua', 'Ut', 'enim', 'ad', 'minim', 'veniam', 'quis', 'quis', 'exercitation']
